In [1]:
%matplotlib widget

# Modelo de Ising 2D como Autômato Celular

Neste notebook implementamos o modelo de Ising bidimensional como um autômato celular.
Cada célula da rede representa um spin que pode assumir os valores +1 ou −1.

A dinâmica do sistema é governada por regras locais que dependem:
- da soma dos quatro vizinhos (vizinhança de von Neumann),
- da constante de acoplamento J,
- do campo magnético externo H,
- da temperatura T.

Para T = 0, a evolução é determinística.
Para T > 0, a evolução é probabilística, modelando a termalização térmica.

As atualizações são feitas de forma assíncrona e são aplicadas diretamente sobre a rede.


In [4]:
from ipywidgets import interact, IntSlider, FloatSlider
import matplotlib.pyplot as plt
import numpy as np


def initialize_lattice(L):
    return np.random.choice([-1, 1], size=(L, L))


def compute_local_field(lattice, i, j, J, H):
    L = lattice.shape[0]
    up    = lattice[(i-1) % L, j]
    down  = lattice[(i+1) % L, j]
    left  = lattice[i, (j-1) % L]
    right = lattice[i, (j+1) % L]
    return J * (up + down + left + right) + H


def ising_step(lattice, J, H, T):
    L = lattice.shape[0]

    for i in range(L):
        for j in range(L):
            h = compute_local_field(lattice, i, j, J, H)

            if T == 0:
                if abs(h) > 1e-12:
                    lattice[i, j] = np.sign(h)
            else:
                beta = 1.0 / T
                r = 1.0 / (1.0 + np.exp(-2 * beta * h))
                lattice[i, j] = 1 if np.random.rand() < r else -1

    return lattice


def magnetization(lattice):
    return np.mean(lattice)


In [6]:
def run_simulation(L, J, H, T, steps):
    lattice = initialize_lattice(L)
    mag_history = []

    for _ in range(steps):
        lattice = ising_step(lattice, J, H, T)
        mag_history.append(magnetization(lattice))

    mag_history = np.array(mag_history)
    mag_mean = np.mean(mag_history)

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # Configuração final
    axes[0].imshow(lattice, cmap='gray')
    axes[0].set_title("Configuração final dos spins")
    axes[0].set_xticks([])
    axes[0].set_yticks([])

    # Magnetização
    axes[1].plot(mag_history, lw=1)
    axes[1].axhline(mag_mean, color='red', linestyle='--',
                    label=f'⟨m⟩ = {mag_mean:.3f}')
    axes[1].set_xlabel("Tempo")
    axes[1].set_ylabel("Magnetização")
    axes[1].legend()

    plt.tight_layout()
    plt.show()


In [5]:
interact(
    run_simulation,
    L=IntSlider(value=50, min=20, max=120, step=10),
    J=FloatSlider(value=1.0, min=-2.0, max=2.0, step=0.1),
    H=FloatSlider(value=0.0, min=-2.0, max=2.0, step=0.1),
    T=FloatSlider(value=2.0, min=0.0, max=5.0, step=0.1),
    steps=IntSlider(value=300, min=50, max=800, step=50)
)


interactive(children=(IntSlider(value=50, description='L', max=120, min=20, step=10), FloatSlider(value=1.0, d…

<function __main__.run_simulation(L, J, H, T, steps)>